# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with campaigns <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover each campaign in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the LastFM dataset.

Start by loading in the dataset which we can use for our lookup table.

In [4]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)


By defining the ID column as the index column it is trivial to return an artist by just querying the ID.

In [5]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

Terminator 2: Judgment Day (1991)


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [6]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [7]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

Princess Bride, The (1987)
Error obtaining title
Error obtaining title


Great! Now we have a way of rendering results. 

### SIMS

SIMS requires just an item as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is a movie. 

The cells below will handle getting recommendations from SIMS and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Terminator 2: Judgment Day).

In [8]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [9]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Jurassic Park (1993)
Braveheart (1995)
Terminator, The (1984)
Fugitive, The (1993)
Speed (1994)
Crimson Tide (1995)
GoldenEye (1995)
Batman (1989)
Clear and Present Danger (1994)
True Lies (1994)
Mask, The (1994)
Die Hard: With a Vengeance (1995)
In the Line of Fire (1993)
Lion King, The (1994)
Ghost (1990)
Forrest Gump (1994)
Apollo 13 (1995)
Cliffhanger (1993)
Star Trek: Generations (1994)
Firm, The (1993)
Die Hard (1988)
Seven (a.k.a. Se7en) (1995)
Indiana Jones and the Last Crusade (1989)
Mission: Impossible (1996)
Mrs. Doubtfire (1993)


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for our sample collection of artists render in a nice dataframe. Again, let's create a helper function to achieve this.

In [10]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Now, let's test the helper function with several different movies. Let's sample some data from our dataset to test our SIMS campaign. Grab 5 random movies from our dataframe.

Note: We are going to show similar titles, so you may want to re-run the sample until you recognize some of the movies listed

In [16]:
samples = items_df.sample(5)
samples

,title
movieId,
1911,Dr. Dolittle (1998)
8451,Blackboard Jungle (1955)
971,Cat on a Hot Tin Roof (1958)
3099,Shampoo (1975)
2944,"Dirty Dozen, The (1967)"


In [17]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

,Dr. Dolittle (1998),Blackboard Jungle (1955),Cat on a Hot Tin Roof (1958),Shampoo (1975),"Dirty Dozen, The (1967)"
0,A Bad Moms Christmas (2017),"Shawshank Redemption, The (1994)","Thomas Crown Affair, The (1968)","Man and a Woman, A (Un homme et une femme) (1966)","African Queen, The (1951)"
1,Jim & Andy: The Great Beyond (2017),Forrest Gump (1994),"Ox-Bow Incident, The (1943)","Morning After, The (1986)",Mo' Better Blues (1990)
2,O.J.: Made in America (2016),Pulp Fiction (1994),"Dolce Vita, La (1960)",52 Pick-Up (1986),Midnight Run (1988)
3,Gaga: Five Foot Two (2017),"Silence of the Lambs, The (1991)","Long, Hot Summer, The (1958)",What Ever Happened to Baby Jane? (1962),"Magnificent Seven, The (1960)"
4,Ice Guardians (2016),Braveheart (1995),Von Ryan's Express (1965),Meatballs (1979),Running Scared (1986)
5,Kurt & Courtney (1998),"Matrix, The (1999)",Requiem for a Heavyweight (1962),"Kentucky Fried Movie, The (1977)",Night Shift (1982)
6,Kevin Hart: Laugh at My Pain (2011),Schindler's List (1993),"Suddenly, Last Summer (1959)",Everything You Always Wanted to Know About Sex...,Logan's Run (1976)
7,"Nobody Speak: Hulk Hogan, Gawker and Trials of...",Star Wars: Episode IV - A New Hope (1977),"Run of the Country, The (1995)",High Art (1998),"Gods Must Be Crazy, The (1980)"
8,Making a Murderer (2015),Jurassic Park (1993),Stand and Deliver (1988),Less Than Zero (1987),Hatchet III (2013)
9,Blue Crush (2002),Terminator 2: Judgment Day (1991),Turtle Diary (1985),Afterglow (1997),Stingray Sam (2009)


You may notice that a lot of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with the movielens small dataset). This shows that the evaluation metrics should not be the only thing you rely on when evaluating your solution version. So when this happens, what can you do to improve the results?

This is a good time to think about the hyperparameters of the Personalize recipes. The SIMS recipe has a `popularity_discount_factor` hyperparameter (see [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). Leveraging this hyperparameter allows you to control the nuance you see in the results. This parameter and its behavior will be unique to every dataset you encounter, and depends on the goals of the business. You can iterate on the value of this hyperparameter until you are satisfied with the results, or you can start by leveraging Personalize's hyperparameter optimization (HPO) feature. For more information on hyperparameters and HPO tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### User Personalization

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [13]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[115, 518, 252]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [18]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [19]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,115,518,252
0,There's Something About Mary (1998),Star Wars: Episode I - The Phantom Menace (1999),Big Hero 6 (2014)
1,Never Been Kissed (1999),"Ring, The (2002)","King's Speech, The (2010)"
2,American Pie (1999),Saw (2004),Shrek (2001)
3,10 Things I Hate About You (1999),"Game, The (1997)",WALLÂ·E (2008)
4,You've Got Mail (1998),Sin City (2005),The Boss Baby (2017)
5,Shakespeare in Love (1998),Casino Royale (2006),Ratatouille (2007)
6,"Wedding Singer, The (1998)","Incredibles, The (2004)",Arthur Christmas (2011)
7,Analyze This (1999),Air Force One (1997),"Wolf of Wall Street, The (2013)"
8,Austin Powers: The Spy Who Shagged Me (1999),Old Boy (2003),The Emoji Movie (2017)
9,My Best Friend's Wedding (1997),Zombieland (2009),"Incredibles, The (2004)"


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

Now lets apply item filters to see recommendations for one of these users within a genre


In [20]:
def get_new_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

You can see the recommendations for movies within a given genre. Within a VOD application you could create Shelves (also known as rails or carosels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

In [21]:
recommendations_df_shelves = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)
for filter_arn in decade_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)

recommendations_df_shelves

,Film-Noir,Fantasy,Western,Documentary,Comedy,Action,IMAX,1970s
0,Tinker Tailor Soldier Spy (2011),Shrek (2001),Django Unchained (2012),Super Size Me (2004),Big Hero 6 (2014),Big Hero 6 (2014),Inception (2010),Taxi Driver (1976)
1,Drive (2011),The Lego Movie (2014),For a Few Dollars More (Per qualche dollaro in...,In the Realms of the Unreal (2004),Shrek (2001),"Incredibles, The (2004)",Tangled (2010),"Godfather, The (1972)"
2,Call Northside 777 (1948),Tangled (2010),Rango (2011),The Green Prince (2014),The Boss Baby (2017),The Lego Movie (2014),Despicable Me 2 (2013),One Flew Over the Cuckoo's Nest (1975)
3,L.A. Confidential (1997),Inside Out (2015),Once Upon a Time in the West (C'era una volta ...,"20,000 Days on Earth (2014)",Arthur Christmas (2011),Inception (2010),How to Train Your Dragon (2010),Star Wars: Episode IV - A New Hope (1977)
4,Sin City (2005),Pinocchio (1940),Butch Cassidy and the Sundance Kid (1969),Planet Earth II (2016),"Wolf of Wall Street, The (2013)",Zootopia (2016),Beauty and the Beast (1991),"Aristocats, The (1970)"
5,Key Largo (1948),"Tale of Princess Kaguya, The (Kaguyahime no mo...",The Hateful Eight (2015),The Salt of the Earth (2014),The Emoji Movie (2017),Dunkirk (2017),Toy Story 3 (2010),Apocalypse Now (1979)
6,"Night of the Hunter, The (1955)",Snow White and the Seven Dwarfs (1937),True Grit (2010),"Honest Liar, An (2014)","Incredibles, The (2004)",Django Unchained (2012),Rise of the Guardians (2012),"Godfather: Part II, The (1974)"
7,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Toy Story 2 (1999),A Million Ways to Die in the West (2014),Why Man Creates (1968),Finding Nemo (2003),Kill Bill: Vol. 2 (2004),Kung Fu Panda (2008),"Clockwork Orange, A (1971)"
8,Double Indemnity (1944),How to Train Your Dragon (2010),Dances with Wolves (1990),Exit Through the Gift Shop (2010),The Lego Movie (2014),Deadpool (2016),"Dark Knight, The (2008)",Watership Down (1978)
9,Gilda (1946),Beauty and the Beast (1991),The Beguiled (2017),13th (2016),Fargo (1996),Brave (2012),Star Wars: Episode VII - The Force Awakens (2015),Annie Hall (1977)


The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [22]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:136455442858:event-tracker/88291ed2
3b3b7d7c-652c-41b4-af91-aef026c8f335


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [23]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [29]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

,252
0,Big Hero 6 (2014)
1,"King's Speech, The (2010)"
2,Shrek (2001)
3,WALLÂ·E (2008)
4,The Boss Baby (2017)
5,Ratatouille (2007)
6,"Wolf of Wall Street, The (2013)"
7,"Incredibles, The (2004)"
8,Finding Nemo (2003)
9,The Lego Movie (2014)


Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [30]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [31]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
    time.sleep(5)

sending event click for Salvador (1986)
sending event click for Die Hard (1988)
sending event click for Power/Rangers (2015)


Now we can look at how the click events changed the recommendations.

In [32]:
user_recommendations_df

,252,Salvador (1986),Die Hard (1988),Power/Rangers (2015)
0,Big Hero 6 (2014),To Kill a Mockingbird (1962),"Lion King, The (1994)",Die Hard (1988)
1,"King's Speech, The (2010)",12 Angry Men (1957),Snatch (2000),"Godfather, The (1972)"
2,Shrek (2001),Citizen Kane (1941),Pinocchio (1940),Indiana Jones and the Last Crusade (1989)
3,WALLÂ·E (2008),"Lion King, The (1994)",Shrek (2001),Amadeus (1984)
4,The Boss Baby (2017),Bonnie and Clyde (1967),Fargo (1996),"Green Mile, The (1999)"
5,Ratatouille (2007),Animal Farm (1954),"Princess Bride, The (1987)","Silence of the Lambs, The (1991)"
6,"Wolf of Wall Street, The (2013)",Amadeus (1984),Home Alone (1990),"Princess Bride, The (1987)"
7,"Incredibles, The (2004)","King's Speech, The (2010)",Toy Story (1995),Shrek (2001)
8,Finding Nemo (2003),Rashomon (RashÃ´mon) (1950),Pokemon 4 Ever (a.k.a. PokÃ©mon 4: The Movie) ...,Apollo 13 (1995)
9,The Lego Movie (2014),"Graduate, The (1967)","Monsters, Inc. (2001)",Goodfellas (1990)


In the cell above, the first column after the index is the user's default recommendations from User Personalization, and each column after that has a header of the artist that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset and effect of a few random clicks. If you wanted to better understand this, try simulating clicking more movies, and you should see a more pronounced impact.

Now lets look at the event filters, which allow you to filter items based on the interaction data. For this dataset, it could be click or watch based on the data we imported, but could be based on whatever interaction schema you design (click, rate, like, watch, purchase etc.) For VOD shelves you could move a title from "Top picks for you" to a "Watch again", the watch again recommendations will be based on the users current interactions, but only recommend titles that have already been watched.


In [33]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Doctor Strange (2016),Deadpool (2016)
1,Rogue One: A Star Wars Story (2016),Guardians of the Galaxy 2 (2017)
2,Star Wars: Episode VII - The Force Awakens (2015),Guardians of the Galaxy (2014)
3,"Incredibles, The (2004)",Logan (2017)
4,Arrival (2016),Thor: Ragnarok (2017)
5,"Lion King, The (1994)",X-Men: Apocalypse (2016)
6,NaN,Edge of Tomorrow (2014)
7,NaN,The Martian (2015)
8,NaN,Aliens (1986)
9,NaN,Avengers: Infinity War - Part I (2018)


now lets send a watch event in for 4 unwatched recommendations, which would simulate watching 4 movies. In a VOD application, you may choose to send in an event after they have watched a significant amount (over 75%) of a piece of content. Sending at 100% complete could miss people that stop short of the credits.

In [34]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)

sending event watch for Deadpool (2016)
sending event watch for Guardians of the Galaxy 2 (2017)
sending event watch for Guardians of the Galaxy (2014)
sending event watch for Logan (2017)


Now we can look at the event filters to see the updated watched and unwatched recommendations 

In [36]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Logan (2017),Blade Runner 2049 (2017)
1,Arrival (2016),Edge of Tomorrow (2014)
2,Guardians of the Galaxy 2 (2017),Deadpool 2 (2018)
3,Big Hero 6 (2014),Lucy (2014)
4,Rogue One: A Star Wars Story (2016),Thor: Ragnarok (2017)
5,Doctor Strange (2016),"Maze Runner, The (2014)"
6,NaN,Dawn of the Planet of the Apes (2014)
7,NaN,Avengers: Infinity War - Part I (2018)
8,NaN,Star Wars: The Last Jedi (2017)
9,NaN,Suicide Squad (2016)


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period etc). This may not be information that you have in your metadata, so a item metadata filter will not work, howeverr you may have this information within you system to generate the item list. 

To demonstrate this, we will use the same user from before and a random collection of items.

In [37]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [38]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

,Un-Ranked
0,"Thrill of It All, The (1963)"
1,Fruitvale Station (2013)
2,Ice Age 2: The Meltdown (2006)
3,Escape from New York (1981)
4,Pulp Fiction (1994)
5,Hurricane Streets (1997)
6,"Marine, The (2006)"
7,The Revenant (2015)
8,Snow White and the Huntsman (2012)
9,Into the Abyss (2011)


Then make the personalized ranking API call.

In [39]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [40]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,Un-Ranked,Re-Ranked
0,"Thrill of It All, The (1963)",The Revenant (2015)
1,Fruitvale Station (2013),Pulp Fiction (1994)
2,Ice Age 2: The Meltdown (2006),AVP: Alien vs. Predator (2004)
3,Escape from New York (1981),Wreck-It Ralph (2012)
4,Pulp Fiction (1994),Escape from New York (1981)
5,Hurricane Streets (1997),Fruitvale Station (2013)
6,"Marine, The (2006)",Yes Man (2008)
7,The Revenant (2015),Stigmata (1999)
8,Snow White and the Huntsman (2012),Chocolat (1988)
9,Into the Abyss (2011),Zookeeper (2011)


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user, a list of promotions for example.

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Recently, Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the HRNN solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for HRNN, it looks like:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [41]:
# We will use the same users from before
users
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [42]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "115"}
{"userId": "518"}
{"userId": "252"}


Upload the file to S3 and save the path as a variable for later.

In [43]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://136455442858-us-east-1-personalizepocvod/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [44]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://136455442858-us-east-1-personalizepocvod/


Now just make the call to kick off the batch export process.

In [45]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 30 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [46]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  10:35:52 AM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInfer

In [47]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

,User: 115,User: 518,User: 252
0,There's Something About Mary (1998),Star Wars: Episode I - The Phantom Menace (1999),Big Hero 6 (2014)
1,Never Been Kissed (1999),"Ring, The (2002)","King's Speech, The (2010)"
2,American Pie (1999),Saw (2004),Shrek (2001)
3,10 Things I Hate About You (1999),"Game, The (1997)",WALLÂ·E (2008)
4,You've Got Mail (1998),Sin City (2005),The Boss Baby (2017)
5,Shakespeare in Love (1998),Casino Royale (2006),Ratatouille (2007)
6,"Wedding Singer, The (1998)","Incredibles, The (2004)","Wolf of Wall Street, The (2013)"
7,Analyze This (1999),Air Force One (1997),"Incredibles, The (2004)"
8,Austin Powers: The Spy Who Shagged Me (1999),Old Boy (2003),Finding Nemo (2003)
9,My Best Friend's Wedding (1997),Zombieland (2009),The Lego Movie (2014)


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `06_Clean_Up_Resources.ipynb`.

In [48]:
%store event_tracker_arn
%store batchInferenceJobArn

Stored 'event_tracker_arn' (str)
Stored 'batchInferenceJobArn' (str)
